In [28]:
import os
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, GlobalAvgPool2D, GlobalMaxPool2D
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.utils import plot_model
from keras.utils import np_utils
import cv2
import math
import numpy as np
import pandas as pd
import scikitplot
import seaborn as sns
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [29]:
data = '/kaggle/input/fer13-cleaned-dataset'

In [30]:
images = 0
for a in os.listdir(data):
    c = 0
    for f in os.listdir(data+'/'+a):
        images += 1
        c += 1
print(f'total images are {images}')
print(c)

total images are 16876
6834


In [31]:
image_array = np.empty(shape = (16876,48,48,3))
image_label = np.empty(shape = 16876)

In [32]:
image_array[1].shape

(48, 48, 3)

In [33]:
#/kaggle/input/fer13-cleaned-dataset/Happy/Training_10000426.jpg
i = 0
lb = 0
for a in os.listdir(data):
    for f in os.listdir(data+'/'+a):
        image_array[i] = cv2.imread(data+'/'+a+'/'+f)
        image_label[i] = lb
        i += 1
    lb += 1

In [34]:
for a in os.listdir(data):
    print(a)

fear
Angry
Neutral
disgust
Happy


In [35]:
image_array.shape,image_label.shape

((16876, 48, 48, 3), (16876,))

In [36]:
label_to_text = {1: 'Angry', 4: 'Happy', 2: 'Neutral', 3: 'disguist',0:'fear'}

In [37]:
label_to_text[2]

'Neutral'

In [38]:
img_label = np_utils.to_categorical(image_label)

In [39]:
img_label.shape

(16876, 5)

In [40]:
img_array = image_array/255

In [41]:
x_train,x_test,y_train,y_test = train_test_split(img_array,img_label,shuffle = True,
                                                stratify = img_label,train_size = 0.9,random_state = 43)

In [42]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((15188, 48, 48, 3), (1688, 48, 48, 3), (15188, 5), (1688, 5))

In [43]:
del img_array
del img_label

In [44]:
neural_net = MobileNet(
    input_shape = (48,48,3),
    include_top = False,
    weights = 'imagenet'
)   
x = neural_net.layers[-14].output
gpool = GlobalMaxPool2D(name = 'global_pool')(x)
outputs = Dense(5,activation = 'softmax',name = 'output_layer')(gpool)
model = Model(inputs = neural_net.input,outputs = outputs)

In [45]:
for layer in model.layers[:15]:
    layer.trainable = False

In [46]:
'''image_generator = ImageDataGenerator(
    rotation_range = 15,
    width_shift_range = 0.15, 
    height_shift_range = 0.15,
    shear_range = 0.15,
    zoom_range = 0.15,
    horizontal_flip = True,
    zca_whitening = True
)
image_generator.fit(x_train)'''

'image_generator = ImageDataGenerator(\n    rotation_range = 15,\n    width_shift_range = 0.15, \n    height_shift_range = 0.15,\n    shear_range = 0.15,\n    zoom_range = 0.15,\n    horizontal_flip = True,\n    zca_whitening = True\n)\nimage_generator.fit(x_train)'

In [47]:
optimizer = optimizers.Adam(0.01)

In [48]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = optimizer,
    metrics = ['accuracy']
)

In [49]:
len(x_train)/32

474.625

In [50]:
trained_history = model.fit(
    x_train,y_train,batch_size = 32,
    validation_data = (x_test,y_test),
    steps_per_epoch = len(x_train)/32,
    epochs = 2,
    use_multiprocessing = True
)

Epoch 1/2
474/474 [==============================] - 42s 86ms/step - loss: 1.2343 - accuracy: 0.5706 - val_loss: 1.2385 - val_accuracy: 0.5675
Epoch 2/2
474/474 [==============================] - 39s 83ms/step - loss: 0.9613 - accuracy: 0.6358 - val_loss: 0.9441 - val_accuracy: 0.6558


In [51]:
import matplotlib.pyplot as plt
from PIL import Image 

In [52]:
dataset_path = '/kaggle/input/dmart-christmas-data/dmart facial emotions/dmart facial emotions'
files = []
for a in os.listdir(dataset_path):
    file = dataset_path+'/'+a+''
    files.append(file)

In [53]:
predicted_image = []
predicted_title = []
for i in (os.listdir(dataset_path)):
    img = cv2.imread(dataset_path+'/'+i)
    img = cv2.resize(img,(48,48))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    x = tf.keras.utils.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis = 0)
    c = model.predict(x)
    c = list(c[0])
    cc = []
    label = ('Angry','Happy','Neutral','disguist','fear')
    for i in range(5):
        if c[i] == max(c):
            cc.append(label[i])
    predicted_image.append(x)
    predicted_title.append(cc)


In [54]:
'''for i in range(15):
    plt.subplot(4,4,i+1)
    plt.imshow(predicted_image[i][0])
    plt.title(predicted_title[i])
    plt.axis('off')'''

"for i in range(15):\n    plt.subplot(4,4,i+1)\n    plt.imshow(predicted_image[i][0])\n    plt.title(predicted_title[i])\n    plt.axis('off')"

Loading and running saved model as json

In [55]:
#from keras.models import model_from_json

In [56]:
'''with open('/kaggle/input/my-project-weights-h5json/my_project_weights','r') as file:
    json_structure = file.read()'''

"with open('/kaggle/input/my-project-weights-h5json/my_project_weights','r') as file:\n    json_structure = file.read()"

In [57]:
#json_model = model_from_json(json_structure)

In [58]:
#json_model.summary()

In [59]:
#json_model.load_weights('/kaggle/input/my-project-weights-h5json/my_project_weights as h5 file.h5')

In [60]:
'''json_model.compile(
    loss = 'categorical_crossentropy',
    optimizer = optimizers.Adam(0.001),
    metrics = ['accuracy']
)'''

"json_model.compile(\n    loss = 'categorical_crossentropy',\n    optimizer = optimizers.Adam(0.001),\n    metrics = ['accuracy']\n)"